# Extending ImageJ: Ops

This example illustrates how to create a new Op and runs this Op with ImageJ **OpService**.

In [ ]:
// Behind a firewall? Configure your proxy settings here.
//System.setProperty("http.proxyHost","myproxy.domain")
//System.setProperty("http.proxyPort","8080")

//////////////////////////////////////////////////////////////
// Load ImageJ. This may take some minutes the first time   //
// while ImageJ is installed into ~/.groovy/grapes locally. //
//////////////////////////////////////////////////////////////
@GrabResolver(name='imagej', root='http://maven.imagej.net/content/groups/public/')
@Grab('net.imagej:imagej:2.0.0-rc-58')
import net.imagej.ImageJ
ij = new ImageJ()
println("ImageJ is ready to go.")

In [ ]:
import net.imagej.ImageJ
import net.imagej.ops.AbstractOp
import net.imagej.ops.Op
import org.scijava.ItemIO
import org.scijava.plugin.Parameter
import org.scijava.plugin.Plugin
import org.scijava.plugin.PluginInfo

// The @Plugin annotation allows this Op to be discovered by the OpService.
// We declare the type of op, the name of the op, and any optional aliases...
@Plugin(type = Op.class, name = "narf")
public class Narf extends AbstractOp {
  
  // INPUTS, declared using @Parameter notation
  @Parameter
  private String input
  
  // OUTPUTS, declared using @Parameter notation
  @Parameter(type = ItemIO.OUTPUT)
  private String output
  
  @Override
  public void run() {
    // The job of the run method is to populate any outputs using the inputs
    output = "Egads! " + input.toUpperCase()
  }
}

// Create a new ImageJ instance 
ij = new ImageJ()

// The @Plugin annotation is processed by the javac compiler,
// which is used to generate the metadata in class bytecode.
// Unfortunately, the Groovy compiler doesn't invoke the javac
// compiler, so we need to register the plugin manually!
narfInfo = new PluginInfo(Narf.class, Op.class)
ij.plugin().addPlugin(narfInfo)

Now you can start using your new Op.

In [ ]:
// Execute our Op and get the result
String result = ij.op().run("narf", "Put some trousers on")

This example illustrates how to create a new Op that constructs a ramp image.

In [ ]:
import net.imagej.ImageJ
import net.imagej.ops.AbstractOp
import net.imagej.ops.Op
import org.scijava.ItemIO
import org.scijava.plugin.Parameter
import org.scijava.plugin.Plugin
import org.scijava.plugin.PluginInfo
import net.imglib2.Cursor;
import net.imglib2.img.array.ArrayImg
import net.imglib2.img.array.ArrayImgs
import net.imglib2.img.basictypeaccess.array.DoubleArray
import net.imglib2.type.numeric.RealType
import net.imglib2.type.numeric.real.DoubleType

// The @Plugin annotation allows this Op to be discovered by the OpService.
// We declare the type of op, the name of the op, and any optional aliases...
@Plugin(type = Op.class, name = "ramp")
public class Ramp<T extends RealType<T>> extends AbstractOp {
  
  // OUTPUTS, declared using @Parameter notation
  @Parameter(type = ItemIO.OUTPUT)
  private ArrayImg<DoubleType, DoubleArray> rampImg
  
  
  @Override
  public void run() {
    rampImg = ArrayImgs.doubles(256, 256)
    
    Cursor<DoubleType> c = rampImg.localizingCursor()
    long[] pos = new long[rampImg.numDimensions()]
    
    // Iterate the image and get the each pixel location
    // Every pixel value is assigned its locations sum,
    // so generate the ramp pattern image.
    while (c.hasNext()) {
      c.fwd()
      c.localize(pos)
      c.get().setReal(sum(pos))
    }
  }
  
  // a sum method to be called in our Op
  private float sum(long[] pos) {
    float sum = 0
    for (long p : pos) {
      sum += p
    }
    return sum
  }
}

// Create a new ImageJ instance 
ij = new ImageJ()

// The @Plugin annatation is processed by the javac compiler,
// which is used to generate the metadata in class bytecode.
// Unfortunately, the Groovy compiler doesn't invoke the javac
// compiler, so we need to register the plugin manually!
rampInfo = new PluginInfo(Ramp.class, Op.class)
ij.plugin().addPlugin(rampInfo)

Now it is time to call our newly-constructed Op.

In [ ]:
// Executing our op and get the result
ramp = ij.op().run("ramp")

// Display the ramp image
ij.notebook().display(ramp)

This next example illustrates how to create a new op that constructs a 'random blobs' image.

In [ ]:
import java.util.Random
import net.imagej.ImageJ
import net.imagej.ops.AbstractOp
import net.imagej.ops.Op
import net.imglib2.RandomAccess
import net.imglib2.RandomAccessibleInterval
import net.imglib2.img.array.ArrayImgs
import net.imglib2.type.numeric.RealType
import net.imglib2.util.IntervalIndexer
import net.imglib2.util.Intervals
import org.scijava.ItemIO
import org.scijava.log.LogService
import org.scijava.plugin.Parameter
import org.scijava.plugin.Plugin
import org.scijava.plugin.PluginInfo

// The @Plugin annotation allows this Op to be discovered by the OpService.
// We declare the type of op, the name of the op, and any optional aliases...
@Plugin(type = Op.class, name = "blobs")
public class RandomBlobs<T extends RealType<T>> extends AbstractOp {

  // OUTPUTS, declared using @Parameter notation
  @Parameter(type = ItemIO.OUTPUT)
  private RandomAccessibleInterval<T> image
  
  @Parameter
  private LogService log
  
  @Parameter
  private int blobNum
  
  @Parameter
  private int blobSize
  
  @Parameter
  private int xDim
  
  @Parameter
  private int yDim
  
  @Parameter(required = false)
  private long seed = 0xcafebabe
  
  @Override
  public void run() {
    // produce a XxY float64 array-backed image using the input parameters
    image = ArrayImgs.doubles(xDim, yDim)
    long[] pos = new long[image.numDimensions()]
    
    long[] blobCenter = new long[image.numDimensions()]
    long[] dims = new long[image.numDimensions()]
    image.dimensions(dims)
    
    // get the total number elements of the image
    long total = Intervals.numElements(image)
    
    Random r = new Random(seed)
    
    RandomAccess<T> ra = image.randomAccess(image)
    
    // Iterate to generate each blob
    for (int i = 0; i < blobNum; i++) {
      // generate a random positon in [0, total)
      long index = (long) (r.nextDouble() * total)
      // convert the linear index to the 2-D index
      // For example, index = 59662, dims = [256,256],
      // then blobCenter = [14,233]
      IntervalIndexer.indexToPosition(index, dims, blobCenter)
      
      // For generating current blob, it is necessary to scan
      // the whole image to determine the elements which are
      // locate in the radius of the blobCenter.
      for (int j = 0; j < total; j++) {
        IntervalIndexer.indexToPosition(j, dims, pos)
        double dist = distance(pos, blobCenter)
        if (dist > blobSize) {
          continue
        }
        
        // This element is in the radius of the blobCenter, so it is 
        // assigned with value inversely proportional to the distance.
        // Namely, if the distance is 0.0, then the norm is 1.0; if the
        // distance is blobSize, then the norm is 0.0, and so on.
        ra.setPosition(pos)
        double norm = 1.0 - dist / blobSize
        ra.get().setReal(Math.max(ra.get().getRealDouble(), norm))
      }
    }
  }
  /**
   * Computes distance between the given position and a center point.
   */
  private double distance(long[] pos, long[] center) {
    long sumDistSquared = 0
    for (int d = 0; d < center.length; d++) {
      long dist = pos[d] - center[d]
      sumDistSquared += dist * dist
    }
    return Math.sqrt(sumDistSquared)
  }
}

// Create a new ImageJ instance 
ij = new ImageJ()

// The @Plugin annatation is processed by the javac compiler,
// which is used to generate the metadata in class bytecode.
// Unfortunately, the Groovy compiler doesn't invoke the javac
// compiler, so we need to register the plugin manually!
blobsInfo = new PluginInfo(RandomBlobs.class, Op.class)
ij.plugin().addPlugin(blobsInfo)

Again, it is time to try out our new Op.

In [ ]:
// Executing our op and get the result
blobs = ij.op().run("blobs", 30, 15, 256, 256)

// Display the blobs image
ij.notebook().display(blobs)